In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,RobustScaler
from category_encoders.target_encoder import TargetEncoder

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error,mean_squared_log_error,make_scorer
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from optuna.samplers import TPESampler


import os
from catboost import CatBoostRegressor
from tqdm import tqdm
from sklearn.model_selection import KFold


In [26]:
train = pd.read_csv("train2.csv").drop(['사망자수','중상자수','경상자수','부상자수'],axis=1)
test = pd.read_csv("test(통합본).csv")

In [27]:
def gu(address):
    return address.split()[0]
def dong(address):
    return address.split()[1]
def road_1(x):
    return x.split(' - ')[0]
def road_2(x):
    return x.split(' - ')[1]

In [28]:
for df in [train,test]:
    df['구'] = df['주소'].apply(gu)
    df['동'] = df['주소'].apply(dong)
    df.drop('주소',axis=1,inplace=True)
    df['도로구분1'] = df['도로형태'].apply(road_1)
    df['도로구분2'] = df['도로형태'].apply(road_2)
    df.drop('도로형태',axis=1,inplace=True)

In [29]:
train = train[['요일', '기상상태', '도로구분1','도로구분2', '노면상태', '사고유형', '구', '동', '연휴', '년도', '월', '일', '시간',
       '설치장소', '설치개수', 'CCTV설치대수', '급지구분_1', '급지구분_2', '급지구분_3', 'ECLO']]
test = test[['요일', '기상상태', '도로구분1','도로구분2', '노면상태', '사고유형', '구', '동', '연휴', '년도', '월', '일', '시간',
       '설치장소', '설치개수', 'CCTV설치대수', '급지구분_1', '급지구분_2', '급지구분_3']]

In [30]:
categorical_features = list(train.dtypes[train.dtypes == "object"].index)
display(categorical_features)

for i in categorical_features:
    le = LabelEncoder()
    le=le.fit(train[i]) 
    train[i]=le.transform(train[i])
    
    for case in np.unique(test[i]):
        if case not in le.classes_: 
            le.classes_ = np.append(le.classes_, case) 
    test[i]=le.transform(test[i])

['요일', '기상상태', '도로구분1', '도로구분2', '노면상태', '사고유형', '구', '동']

In [31]:
train_x = train.drop('ECLO',axis=1)
train_y = np.log(train['ECLO'])
test_x = test

In [32]:
rs = RobustScaler()
X_standard = rs.fit_transform(train_x)
test_standard = rs.transform(test_x.iloc[:,:])

In [33]:
X_train, X_valid , y_train , y_valid  = train_test_split(X_standard ,train_y, shuffle=True,test_size=0.2)

In [34]:
import numpy as np

def rmsle(y_true, y_pred, squared=True):
    assert len(y_true) == len(y_pred), "Input arrays must have the same length."

    y_true_log1p = np.log1p(y_true)
    y_pred_log1p = np.log1p(y_pred)

    if squared:
        return np.mean((y_true_log1p - y_pred_log1p) ** 2)
    else:
        return np.sqrt(np.mean((y_true_log1p - y_pred_log1p) ** 2))

def objective(trial, train_x, train_y, val_x, val_y):
    param = {
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 0.1),
        'subsample': trial.suggest_float('subsample', 0.1, 1.0),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth',4,15),
    }
    model = CatBoostRegressor(**param)
    model.fit(train_x, train_y)
    preds = model.predict(val_x)
    score = rmsle(val_y, preds, squared=False)
    return score




In [35]:
study =  optuna.create_study(study_name='Catboost', direction='minimize',sampler=TPESampler(seed=42) )
study.optimize(lambda trial: objective(trial,X_train,  y_train ,X_valid , y_valid ),n_trials=50, timeout = 1000 )
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

[I 2023-12-09 16:11:54,313] A new study created in memory with name: Catboost


0:	learn: 3.2252738	total: 10.6ms	remaining: 6.55s
1:	learn: 3.2198103	total: 20ms	remaining: 6.17s
2:	learn: 3.2154793	total: 29.2ms	remaining: 5.99s
3:	learn: 3.2111195	total: 37.7ms	remaining: 5.8s
4:	learn: 3.2080135	total: 46.2ms	remaining: 5.67s
5:	learn: 3.2050730	total: 53.9ms	remaining: 5.51s
6:	learn: 3.2024472	total: 62.3ms	remaining: 5.45s
7:	learn: 3.2003389	total: 69.9ms	remaining: 5.34s
8:	learn: 3.1982198	total: 78ms	remaining: 5.29s
9:	learn: 3.1964666	total: 87.4ms	remaining: 5.32s
10:	learn: 3.1949656	total: 95.9ms	remaining: 5.3s
11:	learn: 3.1932881	total: 105ms	remaining: 5.32s
12:	learn: 3.1917091	total: 113ms	remaining: 5.29s
13:	learn: 3.1903706	total: 122ms	remaining: 5.29s
14:	learn: 3.1894106	total: 131ms	remaining: 5.26s
15:	learn: 3.1881456	total: 138ms	remaining: 5.22s
16:	learn: 3.1871120	total: 147ms	remaining: 5.2s
17:	learn: 3.1863882	total: 155ms	remaining: 5.17s
18:	learn: 3.1850628	total: 163ms	remaining: 5.14s
19:	learn: 3.1843937	total: 171ms	rem

162:	learn: 3.1111137	total: 1.22s	remaining: 3.42s
163:	learn: 3.1106101	total: 1.23s	remaining: 3.41s
164:	learn: 3.1101674	total: 1.24s	remaining: 3.4s
165:	learn: 3.1098349	total: 1.24s	remaining: 3.39s
166:	learn: 3.1093984	total: 1.25s	remaining: 3.38s
167:	learn: 3.1088838	total: 1.26s	remaining: 3.37s
168:	learn: 3.1083853	total: 1.26s	remaining: 3.36s
169:	learn: 3.1080552	total: 1.27s	remaining: 3.36s
170:	learn: 3.1075906	total: 1.28s	remaining: 3.35s
171:	learn: 3.1070056	total: 1.28s	remaining: 3.34s
172:	learn: 3.1065707	total: 1.29s	remaining: 3.33s
173:	learn: 3.1061011	total: 1.3s	remaining: 3.32s
174:	learn: 3.1058077	total: 1.31s	remaining: 3.31s
175:	learn: 3.1051075	total: 1.31s	remaining: 3.31s
176:	learn: 3.1047204	total: 1.32s	remaining: 3.3s
177:	learn: 3.1043142	total: 1.33s	remaining: 3.29s
178:	learn: 3.1038599	total: 1.33s	remaining: 3.28s
179:	learn: 3.1034950	total: 1.34s	remaining: 3.27s
180:	learn: 3.1031596	total: 1.35s	remaining: 3.26s
181:	learn: 3.1

324:	learn: 3.0318055	total: 2.45s	remaining: 2.22s
325:	learn: 3.0314936	total: 2.46s	remaining: 2.21s
326:	learn: 3.0312235	total: 2.46s	remaining: 2.2s
327:	learn: 3.0308872	total: 2.47s	remaining: 2.19s
328:	learn: 3.0306483	total: 2.48s	remaining: 2.19s
329:	learn: 3.0303177	total: 2.49s	remaining: 2.18s
330:	learn: 3.0298948	total: 2.49s	remaining: 2.17s
331:	learn: 3.0295996	total: 2.5s	remaining: 2.16s
332:	learn: 3.0292727	total: 2.51s	remaining: 2.15s
333:	learn: 3.0289326	total: 2.52s	remaining: 2.15s
334:	learn: 3.0285879	total: 2.52s	remaining: 2.14s
335:	learn: 3.0283080	total: 2.53s	remaining: 2.13s
336:	learn: 3.0280393	total: 2.54s	remaining: 2.12s
337:	learn: 3.0277117	total: 2.55s	remaining: 2.12s
338:	learn: 3.0273592	total: 2.55s	remaining: 2.11s
339:	learn: 3.0270250	total: 2.56s	remaining: 2.1s
340:	learn: 3.0267205	total: 2.57s	remaining: 2.09s
341:	learn: 3.0264642	total: 2.57s	remaining: 2.08s
342:	learn: 3.0260396	total: 2.58s	remaining: 2.08s
343:	learn: 3.0

486:	learn: 2.9845591	total: 3.67s	remaining: 995ms
487:	learn: 2.9843257	total: 3.68s	remaining: 987ms
488:	learn: 2.9841044	total: 3.68s	remaining: 980ms
489:	learn: 2.9837890	total: 3.69s	remaining: 972ms
490:	learn: 2.9835207	total: 3.7s	remaining: 965ms
491:	learn: 2.9832594	total: 3.71s	remaining: 958ms
492:	learn: 2.9829476	total: 3.72s	remaining: 950ms
493:	learn: 2.9826885	total: 3.72s	remaining: 942ms
494:	learn: 2.9824955	total: 3.73s	remaining: 935ms
495:	learn: 2.9822134	total: 3.74s	remaining: 927ms
496:	learn: 2.9819033	total: 3.75s	remaining: 920ms
497:	learn: 2.9816222	total: 3.75s	remaining: 913ms
498:	learn: 2.9813926	total: 3.76s	remaining: 905ms
499:	learn: 2.9811307	total: 3.77s	remaining: 898ms
500:	learn: 2.9807558	total: 3.78s	remaining: 890ms
501:	learn: 2.9805109	total: 3.79s	remaining: 883ms
502:	learn: 2.9802809	total: 3.79s	remaining: 875ms
503:	learn: 2.9800577	total: 3.8s	remaining: 867ms
504:	learn: 2.9799179	total: 3.81s	remaining: 860ms
505:	learn: 2.

...

[I 2023-12-09 16:13:14,242] Trial 5 finished with value: 0.4565881619949031 and parameters: {'l2_leaf_reg': 0.07873242017790835, 'subsample': 0.2797064039425238, 'learning_rate': 0.0514720203975198, 'n_estimators': 613, 'max_depth': 4}. Best is trial 5 with value: 0.4565881619949031.


601:	learn: 3.0953497	total: 3.2s	remaining: 58.6ms
602:	learn: 3.0952304	total: 3.21s	remaining: 53.2ms
603:	learn: 3.0951677	total: 3.21s	remaining: 47.9ms
604:	learn: 3.0950952	total: 3.22s	remaining: 42.6ms
605:	learn: 3.0950501	total: 3.23s	remaining: 37.3ms
606:	learn: 3.0949956	total: 3.23s	remaining: 31.9ms
607:	learn: 3.0947832	total: 3.24s	remaining: 26.6ms
608:	learn: 3.0947335	total: 3.24s	remaining: 21.3ms
609:	learn: 3.0946643	total: 3.25s	remaining: 16ms
610:	learn: 3.0946006	total: 3.25s	remaining: 10.6ms
611:	learn: 3.0944069	total: 3.26s	remaining: 5.32ms
612:	learn: 3.0943528	total: 3.26s	remaining: 0us
0:	learn: 3.2300588	total: 7.47ms	remaining: 7.1s
1:	learn: 3.2268043	total: 739ms	remaining: 5m 50s
2:	learn: 3.2230938	total: 1.42s	remaining: 7m 28s
3:	learn: 3.2192746	total: 2.05s	remaining: 8m 5s
4:	learn: 3.2171492	total: 2.71s	remaining: 8m 32s
5:	learn: 3.2142552	total: 3.35s	remaining: 8m 48s
6:	learn: 3.2098653	total: 3.99s	remaining: 8m 58s
7:	learn: 3.207

[W 2023-12-09 16:13:26,966] Trial 6 failed with parameters: {'l2_leaf_reg': 0.0611469403382424, 'subsample': 0.2534717113185624, 'learning_rate': 0.006598654139229424, 'n_estimators': 952, 'max_depth': 15} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_11772\2668193024.py", line 2, in <lambda>
    study.optimize(lambda trial: objective(trial,X_train,  y_train ,X_valid , y_valid ),n_trials=50, timeout = 1000 )
  File "C:\Users\User\AppData\Local\Temp\ipykernel_11772\3867344226.py", line 23, in objective
    model.fit(train_x, train_y)
  File "C:\Users\User\AppData\Roaming\Python\Python39\site-packages\catboost\core.py", line 5703, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None

21:	learn: 3.1638285	total: 12.3s	remaining: 8m 38s


KeyboardInterrupt: 

In [ ]:
model_cat = CatBoostRegressor(**study.best_trial.params)
kf = KFold(n_splits=5, shuffle=True , random_state=42)

ensemble_predicts= []
scores =[]


for train_idx, val_idx in tqdm(kf.split(X_standard), total=5, desc="Processing folds"):
    X_t, X_val = X_standard[train_idx], X_standard[val_idx]
    y_t, y_val = train_y[train_idx], train_y[val_idx]
    
    # 두 모델 모두 학습
    model_cat.fit(X_t, y_t)
    
    # 각 모델로부터 Validation set에 대한 예측을 평균내어 앙상블 예측 생성
    val_pred = model_cat.predict(X_val)
    
    # Validation set에 대한 대회 평가 산식 계산 후 저장
    scores.append(rmsle(y_val, val_pred))
    
    # test 데이터셋에 대한 예측 수행 후 저장
    model_cat_pred = np.exp(model_cat.predict(test_standard))
    model_cat_pred = np.where(model_cat_pred < 0, 0, model_cat_pred)
    
    ensemble_predicts.append(model_cat_pred)

# K-fold 모든 예측의 평균을 계산하여 fold별 모델들의 앙상블 예측 생성
final_predictions = np.mean(ensemble_predicts, axis=0)

# 각 fold에서의 Validation Metric Score와 전체 평균 Validation Metric Score출력
print("Validation : RMSLE scores for each fold:", scores)
print("Validation : RMSLE:", np.mean(scores))

In [14]:
final_predictions

array([3.84142443, 3.44403305, 5.15275654, ..., 4.3555348 , 4.17960141,
       4.71029388])

In [15]:
submission = pd.read_csv('C:/Users/User/Desktop/대구교통사고예측/sample_submission.csv')
submission['ECLO'] = final_predictions
submission.to_csv('C:/Users/User/Desktop/대구교통사고예측/optuna_cat2.csv',index=False)
submission

,ID,ECLO
0,ACCIDENT_39609,3.841424
1,ACCIDENT_39610,3.444033
2,ACCIDENT_39611,5.152757
3,ACCIDENT_39612,4.068527
4,ACCIDENT_39613,4.952406
...,...,...
10958,ACCIDENT_50567,4.605680
10959,ACCIDENT_50568,4.218216
10960,ACCIDENT_50569,4.355535
10961,ACCIDENT_50570,4.179601


In [22]:
min(final_predictions),max(final_predictions),np.mean(final_predictions)

(2.0027562345033463, 5.996977754134176, 3.9406467058559342)

In [17]:
p = pd.read_csv('C:/Users/User/Desktop/대구교통사고예측/cat_pred_0.4269.csv')

In [18]:
p

,ID,ECLO
0,ACCIDENT_39609,3.861853
1,ACCIDENT_39610,3.451107
2,ACCIDENT_39611,5.133610
3,ACCIDENT_39612,4.464113
4,ACCIDENT_39613,4.625894
...,...,...
10958,ACCIDENT_50567,4.242687
10959,ACCIDENT_50568,4.097948
10960,ACCIDENT_50569,4.295058
10961,ACCIDENT_50570,4.184488


In [21]:
min(p['ECLO']),max(p['ECLO']),np.mean(p['ECLO'])

(2.500139643718016, 5.888244422157497, 4.051512172002897)

In [36]:
xg = pd.read_csv('C:/Users/User/Desktop/대구교통사고예측/optuna_xgb2.csv')
xg

,ID,ECLO
0,ACCIDENT_39609,3.367258
1,ACCIDENT_39610,3.224864
2,ACCIDENT_39611,4.509228
3,ACCIDENT_39612,3.952304
4,ACCIDENT_39613,3.980799
...,...,...
10958,ACCIDENT_50567,3.866031
10959,ACCIDENT_50568,3.967485
10960,ACCIDENT_50569,4.005776
10961,ACCIDENT_50570,3.986940


In [37]:
min(xg['ECLO']),max(xg['ECLO']),np.mean(xg['ECLO'])

(2.4403484, 4.7770643, 3.6671455118945544)

In [39]:
fin = []
for a,b in zip(final_predictions,xg['ECLO']):
    fin.append((a+b)/2)

In [41]:
submission = pd.read_csv('C:/Users/User/Desktop/대구교통사고예측/sample_submission.csv')
submission['ECLO'] = fin
submission.to_csv('C:/Users/User/Desktop/대구교통사고예측/optuna_cat2andxgb2.csv',index=False)
submission

,ID,ECLO
0,ACCIDENT_39609,3.604341
1,ACCIDENT_39610,3.334449
2,ACCIDENT_39611,4.830992
3,ACCIDENT_39612,4.010415
4,ACCIDENT_39613,4.466603
...,...,...
10958,ACCIDENT_50567,4.235856
10959,ACCIDENT_50568,4.092851
10960,ACCIDENT_50569,4.180655
10961,ACCIDENT_50570,4.083271
